In [14]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pickle
from typing import List

In [41]:
spark = SparkSession.builder.appName('Exploracao').getOrCreate()

In [38]:
def fazer_transformacao_camada_prata(lista_assunto: List, lista_data: List):
    spark = SparkSession.builder.appName('Exploracao').getOrCreate()
    for assunto in lista_assunto:
        for data in lista_data:
            df_video_json = spark.read.json(f'../dados/bronze/assunto_{assunto}/{data}/estatisticas/req_video.json')
            df_video_json = df_video_json.select(
                'data_extracao',
                f.explode('items').alias('items')
            )
            df_video_json = df_video_json.select(
                'data_extracao',
                f.col('items.id').alias('ID_VIDEO'),
                f.col('items.snippet.publishedAt').alias('DATA_PUBLICACAO'),
                f.col('items.snippet.channelId').alias('ID_CANAL'),
                f.col('items.snippet.channelTitle').alias('NM_CANAL'),
                f.col('items.snippet.categoryId').alias('ID_CATEGORIA'),
                f.col('items.snippet.title').alias('TITULO_VIDEO'),
                f.col('items.snippet.description').alias('DESCRICAO'),
                f.col('items.snippet.tags').alias('TAGS'),
                f.col('items.contentDetails.duration').alias('DURACAO_VIDEOS'),
                f.col('items.statistics.viewCount').alias('TOTAL_VISUALIZACOES'),
                f.col('items.statistics.likeCount').alias('TOTAL_LIKES'),
                f.col('items.statistics.favoriteCount').alias('TOTAL_FAVORITOS'),
                f.col('items.statistics.commentCount').alias('TOTAL_COMENTARIOS'),
            ).sort(f.col("TOTAL_VISUALIZACOES").desc())
            df_video_json = df_video_json.withColumn('ASSUNTO', f.lit(f'assunto_{assunto}'))
            df_video_json.write.mode('append').parquet('../dados/prata/arquivos_assuntos.parquet')
    spark.stop()

In [16]:
df_video_json = spark.read.json('../dados/bronze/assunto_cities_skylines/extracao_data_*/estatisticas/*.json')
df_video_json.show()

+-------------------+--------------------+--------------------+--------------------+--------+
|      data_extracao|                etag|               items|                kind|pageInfo|
+-------------------+--------------------+--------------------+--------------------+--------+
|2024-01-23 10:37:34|q8i-901LKIxeuVfbo...|[{{false, hd, 2d,...|youtube#videoList...|  {1, 1}|
|2024-01-23 10:37:34|jZJPWIDBL3vKrFDwH...|[{{false, hd, 2d,...|youtube#videoList...|  {1, 1}|
|2024-01-23 10:37:34|_94HWxFQluRa5aGnx...|[{{false, hd, 2d,...|youtube#videoList...|  {1, 1}|
|2024-01-23 10:37:34|ddM0r9gLmEQdDP3xd...|[{{false, sd, 2d,...|youtube#videoList...|  {1, 1}|
|2024-01-23 10:37:34|2v09ZAxp1sCZzTA88...|[{{false, hd, 2d,...|youtube#videoList...|  {1, 1}|
|2024-01-23 10:37:34|ySqXveRJ57S1MIT59...|[{{false, hd, 2d,...|youtube#videoList...|  {1, 1}|
|2024-01-23 10:37:34|W5AFCgNxVHD9KeDFY...|[{{false, hd, 2d,...|youtube#videoList...|  {1, 1}|
|2024-01-23 10:37:35|ukJePRS3ofLnBqGKG...|[{{false, hd, 2d,.

In [17]:
df_video_json = df_video_json.select(
    'data_extracao',
      f.explode('items').alias('items')
)

In [18]:
df_video_json.show()

+-------------------+--------------------+
|      data_extracao|               items|
+-------------------+--------------------+
|2024-01-23 10:37:34|{{false, hd, 2d, ...|
|2024-01-23 10:37:34|{{false, hd, 2d, ...|
|2024-01-23 10:37:34|{{false, hd, 2d, ...|
|2024-01-23 10:37:34|{{false, sd, 2d, ...|
|2024-01-23 10:37:34|{{false, hd, 2d, ...|
|2024-01-23 10:37:34|{{false, hd, 2d, ...|
|2024-01-23 10:37:34|{{false, hd, 2d, ...|
|2024-01-23 10:37:35|{{false, hd, 2d, ...|
|2024-01-23 10:37:35|{{false, hd, 2d, ...|
|2024-01-23 10:37:35|{{false, hd, 2d, ...|
|2024-01-23 10:37:35|{{false, hd, 2d, ...|
|2024-01-23 10:37:35|{{false, hd, 2d, ...|
|2024-01-23 10:37:35|{{false, hd, 2d, ...|
|2024-01-23 10:37:35|{{false, hd, 2d, ...|
|2024-01-23 10:37:35|{{false, hd, 2d, ...|
|2024-01-23 10:37:35|{{false, hd, 2d, ...|
|2024-01-23 10:37:35|{{false, hd, 2d, ...|
|2024-01-23 10:37:36|{{false, hd, 2d, ...|
|2024-01-23 10:37:36|{{false, hd, 2d, ...|
|2024-01-23 10:37:36|{{false, hd, 2d, ...|
+----------

In [19]:
df_video_json = df_video_json.select(
    'data_extracao',
    f.col('items.id').alias('ID_VIDEO'),
    f.col('items.snippet.publishedAt').alias('DATA_PUBLICACAO'),
    f.col('items.snippet.channelId').alias('ID_CANAL'),
    f.col('items.snippet.channelTitle').alias('NM_CANAL'),
    f.col('items.snippet.categoryId').alias('ID_CATEGORIA'),
    f.col('items.snippet.title').alias('TITULO_VIDEO'),
    f.col('items.snippet.description').alias('DESCRICAO'),
    f.col('items.snippet.tags').alias('TAGS'),
    f.col('items.contentDetails.duration').alias('DURACAO_VIDEOS'),
    f.col('items.statistics.viewCount').alias('TOTAL_VISUALIZACOES'),
    f.col('items.statistics.likeCount').alias('TOTAL_LIKES'),
    f.col('items.statistics.favoriteCount').alias('TOTAL_FAVORITOS'),
    f.col('items.statistics.commentCount').alias('TOTAL_COMENTARIOS'),
    
).sort(f.col("TOTAL_VISUALIZACOES").desc())
df_video_json = df_video_json.withColumn('ASSUNTO', f.lit('ASSUNTO'))
df_video_json.show()

+-------------------+-----------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+-------+
|      data_extracao|   ID_VIDEO|     DATA_PUBLICACAO|            ID_CANAL|            NM_CANAL|ID_CATEGORIA|        TITULO_VIDEO|           DESCRICAO|                TAGS|DURACAO_VIDEOS|TOTAL_VISUALIZACOES|TOTAL_LIKES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS|ASSUNTO|
+-------------------+-----------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+-------+
|2024-01-22 16:54:36|5QRCMkPPr2U|2024-01-17T21:00:03Z|UCL5k8vBBo1JOHh7A...|       JulioGameplay|          20|JOGANDO Buckshot ...|buckshot roulette...|[buckshot roulett...|         PT57S|               9909|       

In [39]:
lista_assunto = [
    'power_bi',
    'python_and_dados',
    'cities_skylines',
    'cities_skylines_2',
    'linux',
    'linux_gamming',
    'genshin_impact',
    'zelda'
]

datas = [
    'extracao_data_2024_01_17',
    'extracao_data_2024_01_18',
    'extracao_data_2024_01_19',
    'extracao_data_2024_01_20',
    'extracao_data_2024_01_21',
    'extracao_data_2024_01_22',
    'extracao_data_2024_01_23',
]
fazer_transformacao_camada_prata(lista_assunto=lista_assunto, lista_data=datas)

In [42]:
df_teste = spark.read.parquet('../dados/prata/arquivos_assuntos.parquet')
df_teste.show()

+-------------------+-----------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+--------------------+
|      data_extracao|   ID_VIDEO|     DATA_PUBLICACAO|            ID_CANAL|            NM_CANAL|ID_CATEGORIA|        TITULO_VIDEO|           DESCRICAO|                TAGS|DURACAO_VIDEOS|TOTAL_VISUALIZACOES|TOTAL_LIKES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS|             ASSUNTO|
+-------------------+-----------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+--------------------+
|2024-01-23 10:37:43|BVHN3nK8JR8|2024-01-19T17:35:48Z|UCMqGy4xIIGs01ZVc...|       Usando Python|          27|Conversão para St...|Diferença entre D...|[python, joao fut...|   

In [43]:
df_teste.select('ASSUNTO').distinct().show(truncate=False)

+-------------------------+
|ASSUNTO                  |
+-------------------------+
|assunto_linux            |
|assunto_power_bi         |
|assunto_genshin_impact   |
|assunto_python_and_dados |
|assunto_linux_gamming    |
|assunto_zelda            |
|assunto_cities_skylines_2|
|assunto_cities_skylines  |
+-------------------------+



# Trends

In [46]:
df_video_json = spark.read.json(f'../dados/bronze/top_brazil/*/top_brazil/*.json')
df_video_json.show()

+-------------------+--------------------+--------------------+--------------------+-------------+---------+-------------+
|      data_extracao|                etag|               items|                kind|nextPageToken| pageInfo|prevPageToken|
+-------------------+--------------------+--------------------+--------------------+-------------+---------+-------------+
|2024-01-17 11:02:08|9nEFoIIx527RJDLmz...|[{{false, hd, 2d,...|youtube#videoList...|       CDIQAA|{50, 200}|         null|
|2024-01-17 11:02:08|IfgR2EhmaLCilwcYV...|[{{false, hd, 2d,...|youtube#videoList...|       CGQQAA|{50, 200}|       CDIQAQ|
|2024-01-17 11:02:09|ibKmOSqwqAhcqX2i4...|[{{false, hd, 2d,...|youtube#videoList...|      CJYBEAA|{50, 200}|       CGQQAQ|
|2024-01-17 11:02:09|mcWg_LkdM0v9gVWYJ...|[{{false, hd, 2d,...|youtube#videoList...|         null|{50, 200}|      CJYBEAE|
|2024-01-17 17:01:11|RJSX0bITJhSUh5GrP...|[{{false, hd, 2d,...|youtube#videoList...|       CDIQAA|{50, 200}|         null|
|2024-01-17 17:0

In [47]:

df_video_json = df_video_json.select(
                'data_extracao',
                f.explode('items').alias('items')
)
df_video_json = df_video_json.select(
                'data_extracao',
                f.col('items.id').alias('ID_VIDEO'),
                f.col('items.snippet.publishedAt').alias('DATA_PUBLICACAO'),
                f.col('items.snippet.channelId').alias('ID_CANAL'),
                f.col('items.snippet.channelTitle').alias('NM_CANAL'),
                f.col('items.snippet.categoryId').alias('ID_CATEGORIA'),
                f.col('items.snippet.title').alias('TITULO_VIDEO'),
                f.col('items.snippet.description').alias('DESCRICAO'),
                f.col('items.snippet.tags').alias('TAGS'),
                f.col('items.contentDetails.duration').alias('DURACAO_VIDEOS'),
                f.col('items.statistics.viewCount').alias('TOTAL_VISUALIZACOES'),
                f.col('items.statistics.likeCount').alias('TOTAL_LIKES'),
                f.col('items.statistics.favoriteCount').alias('TOTAL_FAVORITOS'),
                f.col('items.statistics.commentCount').alias('TOTAL_COMENTARIOS'),
            ).sort(f.col("TOTAL_VISUALIZACOES").desc())
df_video_json.write.mode('append').parquet('../dados/prata/arquivos_assuntos_trends.parquet')

In [48]:
df_video_json.show()

+-------------------+-----------+--------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+
|      data_extracao|   ID_VIDEO|     DATA_PUBLICACAO|            ID_CANAL|          NM_CANAL|ID_CATEGORIA|        TITULO_VIDEO|           DESCRICAO|                TAGS|DURACAO_VIDEOS|TOTAL_VISUALIZACOES|TOTAL_LIKES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS|
+-------------------+-----------+--------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+---------------+-----------------+
|2024-01-22 11:05:09|kBnouFo40Yc|2024-01-18T00:19:35Z|UC_oToDrJ6uca7d1d...|        Canal GOAT|          17|BOTAFOGO X MADURE...|#botafogo  #madur...|[campeonato cario...|    PT2H42M37S|             995111|      54684|              0|           